In [1]:
class Graph:
    pass

import typing as t
from abc import ABC

class User:
    pass

class UserRepo(ABC):
    pass

class SQLAUserRepo(UserRepo):
    pass

class InMemUserRepo(UserRepo):
    pass

# Touch Typing

Alec Reiter | PyAtl  January 2019

@just_anr http://justanr.github.io

## Let's not start a flame war...

- Static vs Dynamic
- Both have benefits

## A brief history of Type Annotations

* Added in 3.0 (PEP 3107)
* Codified in 3.5 (PEP 484)
* Extended in 3.6 (PEP 526)

## Static Typing for Dynamic Python

* Gradual
* Not checked by interpreter
* Preserved at runtime

## Benefits

* Bug Smasher
* Refactor Safely
* Automatic Documentation
* Metaprogramming Dream

In [4]:
def wrap_in_parens(a):
    return '(' + a.strip() + ')'

In [5]:
print(wrap_in_parens('a'))

(a)


In [6]:
print(wrap_in_parens(1))

AttributeError: 'int' object has no attribute 'strip'

## What's it look like?

## Function Types

In [7]:
def wrap_in_parens(a: str) -> str:
    return '(' + a.strip() + ')'

In [ ]:
wrap_in_parens(1)

`Argument 1 to "wrap_in_parens" has incompatible type "int"; expected "str"`

## lambda

- `lambda x: x`
- Can't type hint lambdas

## Variable Types

In [8]:
a: int = 1

## Class Types

In [9]:
class Point:
    x: int
    y: int

## Collections

In [11]:
def plot_points(points: list) -> Graph:
    ...

... oh that's not good

## Typing Module

* Added in 3.5
* More complex types
* Generics, ClassVar, NewType, Union...

## Collections

In [12]:
def plot_points(points: t.List[Point]) -> Graph:
    ...

## A "New Type"

In [13]:
UserId = t.NewType('UserId', int)

In [15]:
def find_user(id: UserId) -> User:
    ...

In [ ]:
find_user(1)

`Argument 1 to "find_user" has incompatible type "int"; expected "UserId"`

In [17]:
find_user(UserId(1))

## Generics

In [16]:
T = t.TypeVar('T')

class SearchResult(t.Generic[T]):
    results: t.List[T]
    has_more: bool

## Callable Types

In [ ]:
t.Callable[[int, int], int]

## Aliases

In [ ]:
Callable[[AbstractConverter, Tuple[type], Dict[str, Any]], FieldABC]

In [ ]:
FieldFactory = Callable[[AbstractConverter, Tuple[type], Dict[str, Any]], FieldABC]

## Forward References

In [47]:
class LinkedList(t.Generic[T]):
    value: T
    next: t.Optional['LinkedList[T]']

### Careful...

In [ ]:
ll = LinkedList[int](1, LinkedList[int]('hello', None))

## What about Python 2?

- Just upgrade
- It's that easy right?

### Stub Files

- "Header" files
- Python 2 and 3 compat
- Add types to untyped modules
- Typeshed repo

In [ ]:
# unittest.pyi
class TestSuite(Testable):
    def __init__(self, tests: Iterable[Testable] = None) -> None: ...
    def addTest(self, test: Testable) -> None: ...
    def addTests(self, tests: Iterable[Testable]) -> None: ...
    def run(self, result: TestResult) -> None: ...
    def debug(self) -> None: ...
    def countTestCases(self) -> int: ...

## AntiPatterns and Fixing Them

## Not Specific Enough

In [27]:
class SearchResults:
    results: t.List[object]

### Fixing

In [28]:
T = TypeVar("T")

class SearchResults(t.Generic[T]):
    results: t.List[T]

## Too Specific

In [29]:
def add_two(x: int, y: int) -> int:
    return x + y

### Fixing

In [30]:
from typing import TypeVar
from decimal import Decimal

AnyNum = TypeVar('AnyNum', int, float, complex, Decimal)

def add_two(x: AnyNum, y: AnyNum) -> AnyNum:
    return x + y

```
add_two(1, 1)
add_two(1.1, 1.1)
add_two(1j, 1j)
add_two(Decimal(1), Decimal(1))
```

In [32]:
add_two(1, Decimal(1))

Decimal('2')

`error: Value of type variable "AnyNum" of "add_two" cannot be "object"`

## Using Any, object, type

In [42]:
def frob(x: t.Any, y: t.Any, z: t.Any) -> t.Any:
    ...

### Fixing

- Fill in the right types
- _Sometimes_ these are right
- `type` -> `Type[C]`
- `object` -> `C`

## Typing Everything

- Inference
- Biting off too much at once
- _Sometimes_ untyped non-public is more helpful

## Cool, but it's not enforce at runtime...

## Enter mypy

* Offline type checker
* Think "flake8" for types
* https://mypy.readthedocs.io

### Configuring

- tons of settings
- cli arguments
- mypy.ini

### Example

```
[mypy]

ignore_missing_imports = False
show_column_numbers = True
show_error_context = False
follow_imports = normal
cache_dir = /dev/null
disallow_untyped_calls = False
warn_return_any = True
strict_optional = True
warn_no_return = True
warn_redundant_casts = True
warn_unused_ignores = True
disallow_untyped_defs = False
check_untyped_defs = True
```

```
[mypy-marshmallow.*]
ignore_missing_imports = True
```

## As part of your build

```
# tox.ini
[testenv:types]
skip_install = false
deps = -r{toxinidir}/requirements/requirements-types.txt
commands = mypy ./src/marshmallow_annotations
```

```
# .travis.yml

matrix:
  include:
    - python: 3.6
      env: TOXENV=types
```

# Using Types Programatically

## Your new best friend

In [50]:
t.get_type_hints(Point)

{'x': int, 'y': int}

## Lots of things we can do

- Serialization Formats
- Configuration
- Dependency Injection

### Injector

- IoC Container
- Type hints

In [ ]:
from injector import inject
from myapp.database import Connection

class UserService:
    @inject
    def __init__(self, conn: Connection) -> None:
        self.conn = conn

In [ ]:
from injector import Module, provider
from myapp.database import Connection
from myapp.users import UserService

class AppModule(Module):
    @provider
    def get_connection() -> Connection:
        return Connection(...)
    
    def configure(self, binder) -> None:
        binder.bind(UserService)

In [ ]:
from injector import Injector
from myapp.ioc import AppModule

ioc = Injector([AppModule()])
ioc.get(UserService)

### Serialization Formats

- Issue: Entities and JSON
- Write everything twice?
- Nah

In [ ]:
class Artist:
    id: int
    name: str
    albums: List[Album]

    def __init__(self, id, name, albums=None):
        self.id = id
        self.name = name
        self.albums = albums if albums is not None else []

In [ ]:
class ArtistScheme(Schema):
    id = fields.Integer(required=True, allow_none=False)
    name = fields.String(required=True, allow_none=False)
    albums = fields.Nested('AlbumScheme', many=True, required=True, allow_none=False)

In [ ]:
from marshmallow_annotations import AnnotationSchema
from .music import Artist

class ArtistScheme(AnnotationSchema):
    class Meta:
        target = Artist
        register_as_scheme = True

### Configuration

- Configs have types
- Eval is gross
- Parse via known types

In [2]:
class Config:
    threads: int = 4
    timeout: float
    secrets: t.Dict[str, str]

In [6]:
raw_config = "threads=5\ntimeout=5\nsecrets=hello=world,gutentag=Welt"
print(raw_config)

threads=5
timeout=5
secrets=hello=world,gutentag=Welt


In [7]:
from dotenvtypes import bind_to

c = bind_to(Config, raw_config)

In [8]:
print(c.__class__)
print(c.__dict__) 

<class '__main__.Config'>
{'threads': 5, 'timeout': 5.0, 'secrets': {'hello': 'world', 'gutentag': 'Welt'}}


## The world is your oyster

# Questions?